In [1]:
import json
import mysql.connector

from mysql.connector import Error

In [16]:
def create_connection(config):
    try:
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            print("Соединение с базой данных успешно установлено.")
        return connection
    except Error as e:
        print(f"Ошибка подключения: {e}")
        return None
    
with open("params.json") as f:
    dict_conn = json.load(f)

connection = create_connection(dict_conn)

Соединение с базой данных успешно установлено.


In [17]:
def query_experience_ten_years(connection, n):
    """Запрос информации о врачах со стажем работы не меньше N лет"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT first_name, 
                    last_name, 
                    YEAR(CURDATE()) - YEAR(employment_date)
            FROM doctors
            WHERE YEAR(CURDATE()) - YEAR(employment_date) >= ?;
        """
        cursor.execute(query, (n,))
        result = cursor.fetchall()
        print(f"Врачи со стажем не менее {n} лет:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [19]:
# Задание 1: Врачи со стажем работы >= N лет
query_experience_ten_years(connection, 5)

Врачи со стажем не менее 5 лет:
('Петр', 'Петров', 9)
('Анна', 'Смирнова', 6)
('Петр', 'Петров', 9)
('Петр', 'Петров', 9)


In [20]:
def query_patients_by_doctor(connection, doctor_first_name, doctor_last_name):
    """Запрос информации о пациентах, записанных на прием к указанному врачу"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT p.first_name, p.last_name
            FROM patients p
            INNER JOIN appointment a ON p.patient_id = a.patient_id
            INNER JOIN doctors d ON a.doctor_id = d.doctor_id
            WHERE d.first_name = ? AND d.last_name = ?
            ORDER BY p.last_name, p.first_name;
        """
        cursor.execute(query, (doctor_first_name, doctor_last_name))
        result = cursor.fetchall()
        print(f"Пациенты врача {doctor_first_name} {doctor_last_name}:")
        for row in result:
            print(*row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [21]:
# Задание 2: Пациенты врача с пользовательским вводом

#Анна Смирнова

doctor_first_name = input("Введите имя врача: ")
doctor_last_name = input("Введите фамилию врача: ")
query_patients_by_doctor(connection, doctor_first_name, doctor_last_name)

Пациенты врача  :


In [7]:
def query_healthy_patients(connection):
    """Запрос информации о пациентах со статусом 'здоров'"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT first_name, last_name
            FROM patients p
            INNER JOIN appointment a ON p.patient_id = a.patient_id
            WHERE a.status = 'здоров';
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Здоровые пациенты:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [8]:
# Задание 3: Здоровые пациенты
query_healthy_patients(connection)

Здоровые пациенты:
('Иван', 'Иванов')
('Иван', 'Иванов')
('Иван', 'Иванов')


In [9]:
def query_appointments_by_doctors(connection, date_1, date_2):
    """Запрос информации о количестве приемов пациентов по врачам за интервал date_1 - date_2"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT 
                d.first_name, 
                d.last_name, 
                COUNT(a.appointment_id) AS count
            FROM 
                doctors AS d
            LEFT JOIN 
                appointment AS a ON d.doctor_id = a.doctor_id
                AND a.start_date >= ?
                AND a.start_date < ?
            GROUP BY 
                d.first_name, 
                d.last_name;
        """
        cursor.execute(query, (date_1, date_2))
        result = cursor.fetchall()
        print(f"Количество приемов пациентов по врачам {date_1} - {date_2}")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [10]:
# Задание 4: Количество приемов пациентов по врачам за (date1 - date2)
query_appointments_by_doctors(connection, '2023-01-01', '2023-02-01')

Количество приемов пациентов по врачам 2023-01-01 - 2023-02-01
('Петр', 'Петров', 0)
('Анна', 'Смирнова', 0)


In [11]:
def query_top_specializations(connection):
    """Запрос топ 3 наиболее востребованных специализаций врачей среди пациентов"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT 
                s.title AS specialization, 
                COUNT(a.appointment_id) AS count
            FROM 
                specializations AS s
            JOIN 
                doctors AS d ON s.specialization_id = d.specialization_id
            JOIN 
                appointment AS a ON d.doctor_id = a.doctor_id
            GROUP BY 
                s.title
            ORDER BY 
                count DESC
            LIMIT 3;
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Топ 3 наиболее востребованных специализаций врачей:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [12]:
# Задание 5: Топ 3 востребованных специализаций
query_top_specializations(connection)

Топ 3 наиболее востребованных специализаций врачей:
('Хирург', 4)
('lor', 3)


In [13]:
def query_patients_over_30_multiple_doctors(connection):
    """Запрос информации о пациентах старше 30 лет, которые записывались на прием к нескольким врачам"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT 
                p.first_name, 
                p.last_name, 
                p.birthday, 
                COUNT(a.appointment_id) AS count
            FROM 
                patients AS p
            JOIN 
                appointment AS a ON p.patient_id = a.patient_id
            WHERE 
                TIMESTAMPDIFF(YEAR, p.birthday, CURDATE()) > 30
            GROUP BY 
                p.patient_id
            HAVING 
                COUNT(DISTINCT a.doctor_id) > 1
            ORDER BY 
                p.birthday;
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Пациенты старше 30 лет, записывавшиеся к нескольким врачам:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [14]:
# Задание 6: Пациенты старше 30 лет, посещавшие нескольких врачей
query_patients_over_30_multiple_doctors(connection)

Пациенты старше 30 лет, записывавшиеся к нескольким врачам:
('Иван', 'Иванов', datetime.date(1990, 5, 15), 4)


In [15]:
connection.close()